In [2]:
###########################################
#  First load the dataset
# The dataset is downloaded from: https://rios.mp.fhi.mpg.de/
# Citation: Liu, X., Truppe, S., Meijer, G. et al. The diatomic molecular spectroscopy database. J Cheminform 12, 31 (2020). https://doi.org/10.1186/s13321-020-00433-8
############################################
!wget --trust-server-names https://rios.mp.fhi.mpg.de/export_table.php -O moleculedata.csv

%matplotlib widget
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.special import factorial
from scipy.special import genlaguerre, gamma


df = pd.read_csv('moleculedata.csv')


--2021-09-28 22:05:02--  https://rios.mp.fhi.mpg.de/export_table.php
Resolving rios.mp.fhi.mpg.de (rios.mp.fhi.mpg.de)... 134.76.9.14
Connecting to rios.mp.fhi.mpg.de (rios.mp.fhi.mpg.de)|134.76.9.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘moleculedata.csv’

moleculedata.csv        [  <=>               ] 117.70K   463KB/s    in 0.3s    

2021-09-28 22:05:03 (463 KB/s) - ‘moleculedata.csv’ saved [120523]



In [3]:
df

,Molecule,Electronic state,Reduced mass,Te (cm^{-1}),omega_e (cm^{-1}),omega_ex_e (cm^{-1}),Be (cm^{-1}),alpha_e (cm^{-1}),De (10^{-7}cm^{-1}),Re (\AA),D0 (eV),IP (eV),Reference,Date of reference
0,AgAl,X $^1\Sigma^+$,21.544078,0.0000,254.3400,1.1300,0.127960,0.00076,1.2700,2.472800,1.950000e+00,NaN,"K.P.Huber and G.Herzberg, Molecular Spectra an...",OCT 1974
1,AgAl,B $^1\Sigma^+$,21.544078,27459.1700,199.8500,1.6600,0.117060,0.00112,1.6000,2.585400,NaN,NaN,"K.P.Huber and G.Herzberg, Molecular Spectra an...",OCT 1974
2,AgAl,C $^1\Pi$,21.544078,31744.8000,221.0600,NaN,0.122500,NaN,1.6000,2.525000,NaN,NaN,"K.P.Huber and G.Herzberg, Molecular Spectra an...",OCT 1974
3,AgBi,X $0^+$,71.594911,0.0000,152.1400,0.4050,0.019800,0.00004,0.0135,3.450000,NaN,NaN,"K.P.Huber and G.Herzberg, Molecular Spectra an...",FEB 1975
4,AgBi,X' $1$,71.594911,4185.0000,148.5000,0.5000,0.019900,0.00005,0.0143,3.440000,NaN,NaN,"K.P.Huber and G.Herzberg, Molecular Spectra an...",FEB 1975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,YbF,$X^2\Sigma^+$,17.127500,0.0000,506.6674,2.2451,0.242191,0.00155,2.3880,2.016100,NaN,NaN,"K. L. Dunfield, et al. J. Mol. Spec. 174, 433-...",1995
616,YbF,A$^2\Pi_{1/2}$,17.127500,18788.6502,531.3500,2.7210,0.247629,0.00160,1.9948,1.996000,NaN,NaN,"T. Ma, et al., J. Phys. Chem. A, 113, 8038-8044",2009
617,SrF,$A^2\Pi_{1/2}$,15.622111,15072.0900,509.3800,2.1800,0.253613,0.00156,NaN,2.062729,2.546000e-07,NaN,"John F. Barry, PhD thesis, Yale University, 2013",2013
618,FeH,X $^4\Delta_{7/2}$,0.990100,0.0000,1831.8000,34.8600,NaN,NaN,NaN,NaN,NaN,NaN,"Dulick M. et al, ApJ, 594, 651 (2003)",Sep 2003


# Data cleaning

In [26]:
# Remove the records where the vibrational spectrum constants have NaN

df = df.dropna(subset=['omega_ex_e (cm^{-1})','omega_e (cm^{-1})','Re (\AA)','Te (cm^{-1})','Reduced mass'])
# Drop off the the rows where wexe < 0, which will make fitting Morse potential a problem
df = df[df['omega_ex_e (cm^{-1})']>=0]

print("Number of diatomic Morse potential available to plot:",len(df))
print("Number of diatomic molecules available to plot:", len(df['Molecule'].unique()))

Number of diatomic Morse potential available to plot: 527
Number of diatomic molecules available to plot: 180


# Data visualization

In [6]:
# Create the dropdown list of diatomic molecules available
molecule_options=widgets.Dropdown(
    options=df['Molecule'].unique(),
    value='OH',
    description='Molecule:',
    disabled=False,
)

###########################################################################
# Now define the functions to plot Morse potential. 
# The code is adapted from: https://scipython.com/blog/the-morse-oscillator/
# Major modifications to the code:
# 1. All member variables of the Morse class are stored in atomic units (energy: Hartree, mass: m_e, length: bohr, hbar=1) to simplify calculations and avoid unit conversions
# 2. Reduced mass as input instead of mA and mB
# 3. Plots have eV used for energy unit
# 4. Plot different electronic states of the same molecule on the same figure, label term symbol of the electronic states along the potential
##############################################################################

# Let's convert cm-1 to Eh (Hartree)
wavenumber2Hartree = 4.55633e-6
mp = 1836 # Porton mass is 1836 mass of electron
angstrom2Bohr = 1.88973
hbar = 1
hartree2eV = 27.2114
bohr2angstrom = 0.529177

class Morse:
    """A class representing the Morse oscillator model of a diatomic."""

    def __init__(self, mu, we, wexe, re, Te):
        """Initialize the Morse model for a diatomic molecule.
        Input arguments:
        mu input reduced mass (mass unit: proton mass).
        we, wexe are the Morse parameters (cm-1).
        re is the equilibrium bond length (Angstrom).
        Te is the electronic energy (cm-1).
       
        class member variables:(everything in atomic unit)
        mu, in electron mass (standard in atomic unit)
        we, wexe in Hartree
        re in bohr
        De in hartree
        D0 in hatree
        """
        # First of all, convert everything to atomic unit
        # where hbar = 1, me=1, Eh=1
        self.mu = mu*mp 
        self.we, self.wexe = we*wavenumber2Hartree, wexe*wavenumber2Hartree
        self.re = re*angstrom2Bohr 
        self.Te = Te*wavenumber2Hartree 
        self.De = self.we**2*hbar / 4 / self.wexe

        #  Morse parameters, a and lambda.
        self.a = self.calc_a()
        self.ke = 2*self.De*self.a**2
        self.lam = np.sqrt(2 * self.mu * self.De) / self.a / hbar
        # Maximum vibrational quantum number.
        self.vmax = int(np.floor(self.lam - 0.5))

        self.make_rgrid()
        self.V = self.Vmorse(self.r)

    def make_rgrid(self, n=1000, rmin=None, rmax=None, retstep=False):
        """Make a suitable grid of internuclear separations."""

        self.rmin, self.rmax = rmin, rmax
        if rmin is None:
            # minimum r where V(r)=De on repulsive edge
            self.rmin = self.re - np.log(2) / self.a
        if rmax is None:
            # maximum r where V(r)=f.De
            f = 0.999
            self.rmax = self.re - np.log(1-f)/self.a
        self.r, self.dr = np.linspace(self.rmin, self.rmax, n,
                                      retstep=True)
        if retstep:
            return self.r, self.dr
        return self.r

    def calc_a(self):
        """Calculate the Morse parameter, a.

        Returns the Morse parameter, a, in unit bohr^-1

        """
        return self.we*np.sqrt(self.mu/2.0/self.De)

    def Vmorse(self, r):
        """Calculate the Morse potential, V(r).

        Returns the Morse potential at r (in m) for parameters De
        (in J), a (in m-1) and re (in m).

        """

        return self.De * (1 - np.exp(-self.a*(r - self.re)))**2

    def Emorse(self, v):
        """Calculate the energy of a Morse oscillator in state v.

        Returns the energy of a Morse oscillator parameterized by
        equilibrium vibrational frequency we and anharmonicity
        constant, wexe (both in cm-1).

        """
        vphalf = v + 0.5
        return (self.we * vphalf - self.wexe * vphalf**2)

    def calc_turning_pts(self, E):
        """Calculate the classical turning points at energy E.

        Returns rm and rp, the classical turning points of the Morse
        oscillator at energy E (provided in hatree). rm < rp.

        """

        b = np.sqrt(E / self.De)
        return (self.re - np.log(1+b) / self.a,
                self.re - np.log(1-b) / self.a)

    def calc_psi(self, v, r=None, normed=True, psi_max=1):
        """Calculates the Morse oscillator wavefunction, psi_v.

        Returns the Morse oscillator wavefunction at vibrational
        quantum number v. The returned function is "normalized" to
        give peak value psi_max.

        """

        if r is None:
            r = self.r
        xi = 2 * self.lam * np.exp(-self.a*(r - self.re))
        alpha = 2*(self.lam - v) - 1
        psi = (xi**(self.lam-v-0.5) * np.exp(-xi/2) *
               genlaguerre(v, alpha)(xi))
        psi *= psi_max / np.max(psi)
        return psi

    def calc_psi_Normalized(self, v, xi):
        alpha = 2*(self.lam - v) - 1
        psi = (xi**(self.lam-v-0.5) * np.exp(-xi/2) *
               genlaguerre(v, alpha)(xi))
        Nv = np.sqrt(factorial(v) * (2*self.lam - 2*v - 1) /
                     gamma(2*self.lam - v))
        return Nv * psi

    def plot_V(self, ax, **kwargs):
        """Plot the Morse potential on Axes ax."""

        ax.plot(self.r*bohr2angstrom, (self.V+self.Te)*hartree2eV, **kwargs)

    def get_vmax(self):
        """Return the maximum vibrational quantum number."""

        return int(self.we / 2 / self.wexe - 0.5)

    def draw_Elines(self, vlist, ax, **kwargs):
        """Draw lines on Axes ax representing the energy level(s) in vlist."""

        if isinstance(vlist, int):
            vlist = [vlist]
        for v in vlist:
            E = self.Emorse(v)
            rm, rp = self.calc_turning_pts(E)
            ax.hlines((E+self.Te)*hartree2eV, rm*bohr2angstrom, rp*bohr2angstrom, **kwargs)

    def label_levels(self, vlist, ax):
        if isinstance(vlist, int):
            vlist = [vlist]

        for v in vlist:
            E = self.Emorse(v)
            rm, rp = self.calc_turning_pts(E)
            ax.text(s=r'$v={}$'.format(v), x=rp*bohr2angstrom + 0.6,
                    y=(E+self.Te)*hartree2eV, va='center')
            
    def label_electronstate(self, state_str,ax):
        E = self.Emorse(self.vmax)
        ax.text(s=state_str, x=np.max(self.r)*bohr2angstrom,
                y=(E+self.Te)*hartree2eV-0.2, va='center')

    def plot_psi(self, vlist, ax, r_plot=None, scaling=1, **kwargs):
        """Plot the Morse wavefunction(s) in vlist on Axes ax."""
        if isinstance(vlist, int):
            vlist = [vlist]
        for v in vlist:
            E = self.Emorse(v)*hartree2eV
            if r_plot is None:
                rm, rp = self.calc_turning_pts(E)
                x = self.r[self.r<rp*1.2]
            else:
                x = r_plot
            psi = self.calc_psi(v, r=x, psi_max=self.we/2)
            psi_plot = psi*scaling + (self.Emorse(v)+self.Te)*hartree2eV
            ax.plot(x*bohr2angstrom, psi_plot, **kwargs)
            

##########################################
# Below is the interactive plotting part #
##########################################

fig= plt.figure()
@widgets.interact(molname=molecule_options)
def update(molname):
    fig.clf()
    mol = df[df['Molecule']==molname]
    states=mol['Electronic state'].values
    ax = plt.gca()
    # Set the default color cycle
    colors=['blue', 'red', 'green', 'cyan', 'orange', 'gray', 'purple', 'brown', 'teal', 'coral']
    i_color=0
    for state in states:
        mycolor = colors[i_color]
        i_color+=1
        we = mol[mol['Electronic state']==state]['omega_e (cm^{-1})'].values[0]
        wexe = mol[mol['Electronic state']==state]['omega_ex_e (cm^{-1})'].values[0]
        re = mol[mol['Electronic state']==state]['Re (\AA)'].values[0]
        Te =  mol[mol['Electronic state']==state]['Te (cm^{-1})'].values[0]
        mu = mol[mol['Electronic state']==state]['Reduced mass'].values[0]

        X = Morse(mu, we, wexe, re, Te)
        X.make_rgrid()
        X.V = X.Vmorse(X.r)

        X.plot_V(ax,color=mycolor)

        X.draw_Elines(range(X.vmax), ax,linewidth=0.5, color=mycolor)
        X.label_electronstate(state,ax)
        #X.plot_psi([2, 14], ax, scaling=2, color=COLOUR1)
        #X.label_levels([2, 14], ax)

        ax.set_xlabel(r'$r\;/\mathrm{\AA}$')
        ax.set_ylabel(r'Energy (eV)')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Molecule:', index=119, options=('AgAl', 'AgBi', 'AgBr', 'AgCl', 'A…

In [ ]:
for i in range(0,100,5):
    fig = plt.figure()
    molname = df['Molecule'].unique()[i]
    mol = df[df['Molecule']==molname]
    states=mol['Electronic state'].values
    ax = plt.gca()
    # Set the default color cycle
    colors=['blue', 'red', 'green', 'cyan', 'orange', 'gray', 'purple', 'brown', 'teal', 'coral']
    i_color=0
    for state in states:
        mycolor = colors[i_color]
        i_color+=1
        we = mol[mol['Electronic state']==state]['omega_e (cm^{-1})'].values[0]
        wexe = mol[mol['Electronic state']==state]['omega_ex_e (cm^{-1})'].values[0]
        re = mol[mol['Electronic state']==state]['Re (\AA)'].values[0]
        Te =  mol[mol['Electronic state']==state]['Te (cm^{-1})'].values[0]
        mu = mol[mol['Electronic state']==state]['Reduced mass'].values[0]

        X = Morse(mu, we, wexe, re, Te)
        X.make_rgrid()
        X.V = X.Vmorse(X.r)

        X.plot_V(ax,color=mycolor)

        X.draw_Elines(range(X.vmax), ax,linewidth=0.5, color=mycolor)
        X.label_electronstate(state,ax)
        #X.plot_psi([2, 14], ax, scaling=2, color=COLOUR1)
        #X.label_levels([2, 14], ax)

        ax.set_xlabel(r'$r\;/\mathrm{\AA}$')
        ax.set_ylabel(r'Energy (eV)')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

#         plt.title('Morse potential of ' + molname+'\n')
#         plt.savefig(molname+'-Morse.jpg')

# Data analysis

## For diatomic molecules, what is the typical gap between the vibrational states $v=0$ and $v=1$?

In [62]:
# First create an array of the energy difference between v=0 and v=1 for all records
dE_v = {}
for i in range(0,len(df)):
        row = df.iloc[i]
        we = row['omega_e (cm^{-1})']
        wexe = row['omega_ex_e (cm^{-1})']
        re = row['Re (\AA)']
        Te =  row['Te (cm^{-1})']
        mu = row['Reduced mass']

        X = Morse(mu, we, wexe, re, Te)
        diff = X.Emorse(1)-X.Emorse(0)
        
        mol = row['Molecule']
        state = row['Electronic state']
        dE_v[mol+' '+state]=diff*hartree2eV

### Create a histogram to visualize

In [63]:
n_bins = 20

fig_hist1 = plt.figure()

# We can set the number of bins with the `bins` kwarg
plt.hist(dE_v.values(), bins=n_bins)
plt.xlabel(r'$\Delta$E (eV)')
plt.ylabel('frequency')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'frequency')

### Find the molecules with max/min dE

In [71]:
value_list = list(dE_v.values())
key_list = list(dE_v.keys())
print('min',key_list[value_list.index(min(dE_v.values()))])
print('max',key_list[value_list.index(max(dE_v.values()))])

min PbO B $1$
max HF X $^1\Sigma^+$


## For diatomic molecules, what is the typical gap between the two lowest electronic states?


In [47]:
mol_multiES = []
for mol in df['Molecule'].unique():
    if len(df[df['Molecule']==mol]) > 1:
        mol_multiES.append(mol)

In [38]:
len(df['Molecule'].unique()), len(mol_multiES)

(180, 172)

create a subset of df that only contains those molecules we care about (with multiple electronic states available)

In [59]:
dE_e ={}
for mol in mol_multiES:
    df_mol = df[df['Molecule']==mol]
    Te_mol = df_mol['Te (cm^{-1})'].values
    Te_mol.sort()
    diff = (Te_mol[1]-Te_mol[0])*wavenumber2Hartree*hartree2eV
    dE_e[mol] = diff

### Create a histogram to visualize

In [61]:
n_bins = 20

fig_hist2 = plt.figure()

# We can set the number of bins with the `bins` kwarg
plt.hist(dE_e.values(), bins=n_bins)
plt.xlabel(r'$\Delta$E (eV)')
plt.ylabel('frequency')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'frequency')

### Find the molecules with max/min dE

In [72]:
value_list = list(dE_e.values())
key_list = list(dE_e.keys())
print('min',key_list[value_list.index(min(dE_e.values()))])
print('max',key_list[value_list.index(max(dE_e.values()))])

min TiS
max DF
